# 한메타자 같은 타이핑 게임 만들기 (스마트버전)

* 안내로 제시한 문구를 사용자가 똑같이 입력하여 사용자의 타자속도를 측정하는 게임
* 타자수는 글자수가 아닌 실제 입력한 타이핑의 전체 수로 계산을 해야하므로
* 한글인경우 초성, 중성, 종성을 분리해서 계산해야 더 정확한 속도 측정이 가능합니다.
>한 = ㅎ ㅏ ㄴ = 3타

* 난이도 : ★★★★☆☆☆☆☆☆
* 다루는 내용:
    * 함수, docstring
    * 반복문, 조건문, 사용자 입력, 화면 지우고 대기하기
    * 유니코드 한글 초성, 중성, 종성 분해
    * 랜덤셔플, 리스트
    * 시간측정 하기

### 한글 초성, 중성, 종성 분리


초성 = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']  
중성 = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']  
종성 = [None, 'ㄱ','ㄲ','ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


유니코드 테이블을 보면 위의 순서의 조합으로 모든 글자를 표현하게 되어있습니다.  


**한글 = ((초성 * 21) + 중성) * 28 + 종성 + 0xAC00**    


따라서 이를 역으로 계산하면 해당 음소를 분리하여 각 음소의 인덱스값을 구할 수 있습니다.  

https://ko.wikipedia.org/wiki/%EC%9C%A0%EB%8B%88%EC%BD%94%EB%93%9C_3000~3FFF (자소코드표)  
https://ko.wikipedia.org/wiki/%EC%9C%A0%EB%8B%88%EC%BD%94%EB%93%9C_A000~AFFF (한글코드표)  

In [2]:
#0xAC00 은 '가' 에 해당하는 유니코드 코드값으로 10진수로 44032 입니다. 이를 파이썬에서 테스트 해보면   
# 0번째 초성과 0번째 중성 1번째 종성의 값을 chr 로 구해보면 '각' 이 나옵니다.
chr(((0 * 21) + 0) * 28 + 1 + 44032) == "각"

True

## 초성, 중성, 종성 분리 파이썬 코드

In [3]:
CHO = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
JUNG = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
JONG = ['', 'ㄱ','ㄲ','ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 
        'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

'''
0xAC00 = '가'
한글 = ((초성 * 21) + 중성) * 28 + 종성 + 0xAC00

이를 역으로 계산하면 문자 코드값으로부터 각 음소의 인덱스값을 구할 수 있다.

초성 = ((문자코드 – 0xAC00) / 28) / 21
중성 = ((문자코드 – 0xAC00) / 28) % 21
종성 = (문자코드 – 0xAC00) % 28

여기서 0xAC00 을 10진수로 변환하면 44032 = int("0xac00", 16)
혹은 ord("가") = 44032

초성 인덱스 = (x - 44032) / (21 * 28)
중성 인덱스 = (x - 44032 - (초성 인덱스 * 21 * 28)) / 28
종성 인덱스 = (x - 44032 - (초성 인덱스 * 21 * 28) - (중성 인덱스 * 28))
'''
keyword = input("입력> ")
print(keyword)
keyword_list  = list(keyword)

break_words = []
for k in keyword_list:

    if ord(k) >= ord('가') and ord(k) <= ord('힣'):
        # 유니코드상 몇번째 글자인지 인덱스를 구합니다.
        # ord("각") - ord("가")
        char_index = ord(k) - ord('가')
        
        # 초성 = ((문자코드 – 0xAC00) / 28) / 21
        char1 = int(int(char_index / 28) / 21)
        break_words.append(CHO[char1])

        # 중성 = ((문자코드 – 0xAC00) / 28) % 21
        char2 = int(int(char_index / 28) % 21)
        break_words.append(JUNG[char2])

        # 종성 = (문자코드 – 0xAC00) % 28
        char3 = int(char_index % 28)

        if char3 > 0:
            break_words.append(JONG[char3])
    else:
        break_words.append(k)
        
print("입력: {}".format(keyword))
print("분해: {}".format(break_words))

입력> 한글
한글
입력: 한글
분해: ['ㅎ', 'ㅏ', 'ㄴ', 'ㄱ', 'ㅡ', 'ㄹ']


In [ ]:
import random # 리스트 셔플을 위해 
import time # 시간 측정을 위해
import os # 화면 지우고 대기하기 위해 사용

# 초성, 중성, 종성을 분리 하기 위한 한글 인덱스
# 실제 유니코드 표의 순서대로임
CHO = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
JUNG = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
JONG = ['', 'ㄱ','ㄲ','ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def break_korean(string):
    '''입력된 string을 초성, 중성, 종성으로 분리하는 함수

    Args:
        string (str) : 분해할 문자열
    
    Returns:
        list: 한글이 초 + 중 + 종성으로 분리된 결과 리스트
    '''

    # 입력된 인자를 list 형태로 형변환 한다.
    keyword_list  = list(string)

    # 최종 분해된 결과를 저장할 리스트 변수
    break_words = []

    # 문자열 갯수 만큼 반복
    for k in keyword_list:
        # 한글인 경우에만 처리 ('가' 부터 '힣' 글자 사이의 글자인지를 판단함)
        if ord(k) >= ord('가') and ord(k) <= ord('힣'):
            # 초성 = ((문자코드 – 0xAC00) / 28) / 21
            char_index = ord(k) - ord('가')
            char1 = int(int(char_index / 28) / 21)
            break_words.append(CHO[char1])
            
            # 중성 = ((문자코드 – 0xAC00) / 28) % 21
            char2 = int(int(char_index / 28) % 21)
            break_words.append(JUNG[char2])
            
            # 종성 = (문자코드 – 0xAC00) % 28
            char3 = int(char_index % 28)

            # 마지막 종성은 0이면 아무것도 없는거라 0 이상인 경우에만 추가
            if char3 > 0:
                break_words.append(JONG[char3])
        # 한글이 아닌 경우에는 무조건 추가(영문, 특수문자 등)
        else:
            break_words.append(k)
    
    # 최종 분해 결과 리스트 리턴
    return break_words

# 출제 문장 리스트
WORD_LIST = [
    "남박사의 파이썬 100% 실전 프로그래밍 강좌",
    "파이썬에서 ord() 함수는 문자의 유니코드 값을 알아오는 함수로 10진수 값을 리턴 합니다.",
    "chr(x)은 x에 유니코드 10진수 값을 입력하면 해당하는 문자를 리턴합니다.",
    "UTF-8은 유니코드를 8비트 기반으로 저장하는 인코딩 방식입니다.",
    "CP949는 윈도우에서 사용하기 위해 EUC-KR을 확장해서 만든 문자셋 입니다.",
    "파이썬은 코드가 짧고 유연하여 가독성과 생산성이 좋은 프로그래밍 언어 입니다.",
    "독도는 우리땅"
]

# 문제 리스트 셔플(순서를 섞음)
random.shuffle(WORD_LIST)
current_count = 0

# 문제수 만큼 반복
for q in WORD_LIST:
    # 화면 지움
    os.system("cls")

    # 현재 진행 횟수 1 증가
    current_count += 1

    # 시작 시간 기록
    start_time = time.time()

    # 문제 출제 및 사용자에게 입력 받음
    # 입력받은 데이터는 str 로 형변환 후 앞, 뒤 공백제거 strip()
    user_input = str(input(q + '\n')).strip()
    
    # 최종 입력 직후 진행 시간 계산 (현재시간 - 시작시간)
    end_time = time.time() - start_time

    # 문자 및 사용자 입력 문구 초성, 중성, 종성 분해
    src = break_korean(q)
    tar = break_korean(user_input)

    # 일치하는 문자수 카운팅 변수
    correct = 0

    # 사용자가 입력한 문장을 분해한 문자 갯수 만큼 반복문
    for i, c in enumerate(tar):
        # 현재 인덱스가 원래 문제의 수보다 크면 반복문 탈출 (오류방지)
        if i >= len(src):
            break

        # 문제의 인덱스에 해당하는 글자와 사용자가 입력한 문장의 현재 인덱스에 해당하는 문자가 같으면 
        # 일치하는 문자수 변수를 1 증가
        if src[i] == c:
            correct += 1
    
    # 원래 문제의 분해된 문자 갯수 
    src_len = len(src)

    # 정확도 = 일치하는문자수 / 전체문자수 * 100
    c = correct / src_len * 100

    # 오타율 = (전체문자수 - 일치하는문자수) / 전체문자수 * 100
    e = (src_len - correct) / src_len * 100

    # 속도 = (일치하는문자수 / 입력에 걸린 시간) * 60초 (분당 몇타 이기 때문에 분을 기준)
    speed = float(correct / end_time) * 60

    # 최종 결과 출력
    print("=" * 60)
    print("현재 {}회, 총 글자수: {}, 타수: {}, 입력 타수: {}".format(current_count, len(q), src_len, len(tar)))
    print("속도: {:0.2f} 정확도: {:0.2f} % 오타율: {:0.2f} %".format(speed, c, e))

    # 계속하려면 아무키나 누르세요...
    # 다시 반복문 초기로 돌아가면 바로 시간이 측정되기 때문에 다음 문제 출제전 사용자에게 준비 시간을 줌
    os.system("pause")
